In [ ]:
import numpy as np
pup = np.load("segment_phasing_fp_env/images/pup64.npy")
pup_width = 64
gap_width_m = 0.359
seg_width_m = 8.4

In [ ]:
x0 = pup_width/2
y0 = pup_width/2
seg_diam = pup_width/(3+gap_width_m/seg_width_m*2)
yy,xx = np.mgrid[:pup_width,:pup_width]*1.0
zz = np.tile(xx[None,:,:]*0,[7,1,1])
zz[0,:,:] = (((xx-x0)**2+(yy-y0)**2)**0.5 < seg_diam/2)*1.0
for i,theta in enumerate(np.linspace(0,2*np.pi,7)[:-1]):
    radial_offset = seg_diam*(1+gap_width_m/seg_width_m)
    zz[1+i,:,:] = (((xx-x0-radial_offset*np.cos(theta))**2+(yy-y0-radial_offset*np.sin(theta))**2)**0.5 < seg_diam/2)*1.0
zz *= pup[None,:,:]
zz = zz[1:,...]
zz[:,pup==1] -= zz[:,pup==1].mean(axis=1)[:,None]


In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
fig,ax = plt.subplots(1,6,figsize=[10,2])
i = 0
for a,z in zip(ax,zz):
    a.imshow(z)
    a.set_title(f"mode {i:d}")
    i += 1
    a.set_xticks([])
    a.set_yticks([])
plt.tight_layout()


In [ ]:
np.save("segment_phasing_fp_env/images/modes64.npy",zz[:,pup==1])

In [ ]:
pup_width: int = 64  # width of pupil in pixels
sampling: float = 1.8  # pixels per FWHM of diffraction limited PSF
im_width: int = 20  # width of image

In [ ]:
dft = np.fft.fft(np.eye(int(pup_width*sampling*2))).astype(np.complex64)
dft = dft[:, :pup_width]
dft = np.fft.fftshift(dft, axes=[0])
dft = dft[
    dft.shape[0]//2-im_width:dft.shape[0]//2+im_width, :
]
dft2 = np.kron(dft,dft)[:,pup.flatten()==1]
dft2 /= (np.abs(dft2)**2).sum()**0.5
phi_shift = (np.mgrid[:pup_width,:pup_width]/int(pup_width*sampling*2)).sum(axis=0)[pup==1]
dft2 *= np.exp(-1j*2*np.pi*phi_shift/2)[None,:]

In [ ]:
np.save("segment_phasing_fp_env/images/dft64.npy", dft2)

In [ ]:
im_out = np.abs(dft2.sum(axis=1))**2
plt.matshow(im_out.reshape([40,40]))